In [9]:
# Let us work some circuit, it will be
# I know 4 numbers a1*a2*a3*a4 = 48
# Let us choose values for variables:   a1=6    a2=2   a3=2   a4=2   a5=12    a6=4

# a1*a2=a5
# a3*a4=a6
# a5*a6=a7

import numpy as np
import galois
import random
from py_ecc.bn128 import G1, G2, add, multiply, curve_order, pairing


# a = [1, a7, a1, a2, a3, a4, a5, a6]
# L = [[0,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,0,1,0]]
# R = [[0,0,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1]]
# O = [[0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1],[0,1,0,0,0,0,0,0]]

# I want to check if La * Ra == Oa
a = np.array([1, 48, 6, 2, 2, 2, 12, 4])
L = np.array([[0,0,1,0,0,0,0,0],[0,0,0,0,1,0,0,0],[0,0,0,0,0,0,1,0]])
R = np.array([[0,0,0,1,0,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,1]])
O = np.array([[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1],[0,1,0,0,0,0,0,0]])

np.matmul(L,a) * np.matmul(R,a) == np.matmul(O,a)

array([ True,  True,  True])

In [47]:
##### TRUSTED SETUP #######

GF = galois.GF(curve_order, primitive_element=5, verify=False)
xs = GF([0,1,2])


# INTERPOLATE THE POLYNOMIALS
us = []
for i in L.T:
    u = galois.lagrange_poly(xs,GF(i))
    us.append(u)

vs = []
for i in R.T:
    v = galois.lagrange_poly(xs,GF(i))
    vs.append(v)

ws = []
for i in O.T:
    w = galois.lagrange_poly(xs,GF(i))
    ws.append(w)
    
# SCALARS
# tau, alpha, beta, gamma, delta

tau = GF(random.randint(1, curve_order))
alpha = GF(random.randint(1, curve_order))
beta = GF(random.randint(1, curve_order))
gamma = GF(random.randint(1, curve_order))
delta = GF(random.randint(1, curve_order))

alphaG1 = multiply(G1, int(alpha))
betaG2 = multiply(G2, int(beta))
betaG1 = multiply(G1, int(beta))
gammaG2 = multiply(G2, int(gamma))
deltaG1 = multiply(G1, int(delta))
deltaG2 = multiply(G2, int(delta))

# SRS1 = [G1, tauG1, tau^2G1,...tau^n-1 G1]
# SRS2 = [G2, tauG2, tau^2G2,...tau^n-1 G2]
# SRS3 = [T(tau)/delta G2, T(tau)tau/delta G2, T(tau)tau^2/delta G2,...T(tau)tau^n-2/delta G2]

SRS1 = []
SRS2 = []
SRS3 = []

tau = random.randint(1, curve_order)

T = galois.Poly.Roots(xs,field=GF)
Ttau = int(T(tau))  

for i in range(len(L)):
    point = multiply(G1, tau**i) # generate G1 * tau^i
    SRS1.append(point)  # add to SRS1

for i in range(len(L)):
    point = multiply(G2, tau**i) # generate G2 * tau^i
    SRS2.append(point)  # add to SRS2

for i in range(len(L)-1):
    scalar = T(tau)*(tau**i)/delta
    print(scalar)
    point = multiply(G1, int(scalar)) # generate G1 * T(tau) * tau^i
    SRS3.append(point)  # add to SRS3

# PSI_i_for_verifier = [alpha v(i) + beta u(i) + w(i)]/gamma  i from 0 to 1 
# PSI_i_for_prover = [alpha v(i) + beta u(i) + w(i)]/delta  2 from size of witness

PSI_V_G1 = []
for i in range(2):
    scalar = (alpha*vs[i](tau) + beta*us[i](tau) + ws[i](tau))/gamma
    point = multiply(G1, int(scalar))
    PSI_V_G1.append(point)


PSI_P_G1 = []
for i in range(2,len(a)):
    scalar = (alpha*vs[i](tau) + beta*us[i](tau) + ws[i](tau))/delta
    point = multiply(G1, int(scalar))
    PSI_P_G1.append(point)



15198139725816168642092044182051293986008473739632407355677800532067308103838
5277859716275454832757855809586337933420849323206432041136503270344296482526


In [53]:
## CREATE THE PROOF
# INTERPOLATE THE POLYNOMIALS
us = []
for i in L.T:
    u = galois.lagrange_poly(xs,GF(i))
    us.append(u)

vs = []
for i in R.T:
    v = galois.lagrange_poly(xs,GF(i))
    vs.append(v)

ws = []
for i in O.T:
    w = galois.lagrange_poly(xs,GF(i))
    ws.append(w)

l = np.dot(a,us) # l(x)
r = np.dot(a,vs) # r(x)
o = np.dot(a,ws) # o(x)

p = l*r - o
T = galois.Poly.Roots(xs, field=GF)
h = (l*r - o) // T
remainder = (l*r - o) % T


ltau = None
lcoeffs = (l.coeffs)[::-1]
for i in range(len(SRS1)):
    ltau = add(ltau, multiply(SRS1[i],int(lcoeffs[i])))  # ltau = ltau + SRS[i] * coefs_of_l[i] 

rtau = None
rcoeffs = (r.coeffs)[::-1]
for i in range(len(SRS2)):
    rtau = add(rtau, multiply(SRS2[i],int(rcoeffs[i])))  

rtauG1 = None
rcoeffs = (r.coeffs)[::-1]
for i in range(len(SRS1)):
    rtauG1 = add(rtauG1, multiply(SRS1[i],int(rcoeffs[i])))  
    

htau = None
hcoeffs = (h.coeffs)[::-1]
for i in range(len(SRS3)):
    htau = add(htau, multiply(SRS3[i],int(hcoeffs[i])))

r = GF(random.randint(1, curve_order))
s = GF(random.randint(1, curve_order))

rDeltaG1 = multiply(deltaG1, int(r))
sDeltaG2 = multiply(deltaG2, int(s))

# B = r(tau) G2 + beta G2
# BG1 = r(tau) G1 + beta G1
BG1 = add(rtauG1, betaG1)

A = add(ltau, alphaG1)
B = add(rtau, betaG2)

aPsiProver = None
for i in range(2,len(a)):
    point = multiply(PSI_P_G1[i-2], a[i])
    aPsiProver = add(aPsiProver,point)

C = add(htau, aPsiProver)


# A = l(tau)G1 + alpha G1 + r delta
# B = r(tau)G2 + beta G2 + s delta
# C = h(tau) T(tau) / delta + sum_i=2,len(a) a_i Psi_i / delta + sA + rB - rsDelta


In [54]:
## VERIFICATION PROCESS

# pair(A, B) == pair(alpha, beta) * pair(C, delta) * pair( a_0 Psi_0 / gamma + a_1 Psi_1 / gamma, gamma)

aPsiVerifier = None
for i in range(0,2):
    point = multiply(PSI_V_G1[i], a[i])
    aPsiVerifier = add(aPsiVerifier,point)

pairing(B,A) == pairing(betaG2, alphaG1) * pairing(deltaG2, C) * pairing( gammaG2, aPsiVerifier)


True